In [1]:
import tensorflow as tf
import gc
import os
import numpy as np
import tensorflow_datasets as tfds

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [2]:
!pip install hickle
import hickle as hkl
import os
import matplotlib.pyplot as plt
from keras import layers,Input,Sequential
from keras.layers import Dense,Flatten,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Reshape,Resizing,Dropout,UpSampling2D
from tensorflow.keras import datasets, layers, models, losses, Model
from random import randint
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 1.7 MB/s eta 0:00:00
Mounted at /gdrive
/gdrive


In [ ]:
load_hkl = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Left.hkl")
X_train_left=load_hkl['xtrain'] / 255.
X_test_left=load_hkl['xtest'] / 255.

del load_hkl
gc.collect()

load_hkl = hkl.load("/gdrive/My Drive/Yedek/Autoencoder_Datasets/Static/Right.hkl")
X_train_right=load_hkl['xtrain'] / 255.
X_test_right=load_hkl['xtest'] / 255.

del load_hkl
gc.collect()

X_train_autoencoder=np.concatenate((X_train_left,X_train_right),axis=0)
X_test_autoencoder=np.concatenate((X_test_left,X_test_right),axis=0)
del X_train_right
del X_train_left

In [ ]:
X_train_autoencoder.shape

(376, 480, 640, 3)

In [ ]:
class CNN_Autoencoder(Model):
    def __init__(self):
        super(CNN_Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=X_train_autoencoder.shape[1:]),
            layers.Conv2D(3, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(6, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(12, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(24, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=2),
            layers.Conv2D(48, 3, strides=1, padding='same', activation='relu')])

        self.decoder = tf.keras.Sequential([
            layers.Conv2D(48, 3, strides=1, padding='same', activation='relu'),
            layers.UpSampling2D(2),
            layers.Conv2D(24, 3, strides=1, padding='same', activation='relu'),
            layers.UpSampling2D(2),
            layers.Conv2D(12, 3, strides=1, padding='same', activation='relu'),
            layers.UpSampling2D(2),
            layers.Conv2D(6, 3, strides=1, padding='same', activation='relu'),
            layers.UpSampling2D(2),
            layers.Conv2D(3, 3, strides=1, padding='same', activation='relu')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
with strategy.scope():
  cnn_autoencoder_right_left=CNN_Autoencoder()
  cnn_autoencoder_right_left.compile(optimizer='adam',  loss='mse')
history=cnn_autoencoder_right_left.fit(X_train_autoencoder,X_train_autoencoder,epochs=60,batch_size=64,shuffle=True,validation_data=(X_test_autoencoder,X_test_autoencoder))

Epoch 1/60
6/6 [==============================] - 52s 6s/step - loss: 0.1600 - val_loss: 0.1388
Epoch 2/60
6/6 [==============================] - 4s 650ms/step - loss: 0.1251 - val_loss: 0.1166
Epoch 3/60
6/6 [==============================] - 4s 661ms/step - loss: 0.0938 - val_loss: 0.0535
Epoch 4/60
6/6 [==============================] - 4s 655ms/step - loss: 0.0463 - val_loss: 0.0457
Epoch 5/60
6/6 [==============================] - 4s 719ms/step - loss: 0.0375 - val_loss: 0.0295
Epoch 6/60
6/6 [==============================] - 4s 648ms/step - loss: 0.0289 - val_loss: 0.0266
Epoch 7/60
6/6 [==============================] - 4s 670ms/step - loss: 0.0246 - val_loss: 0.0216
Epoch 8/60
6/6 [==============================] - 4s 661ms/step - loss: 0.0177 - val_loss: 0.0123
Epoch 9/60
6/6 [==============================] - 4s 654ms/step - loss: 0.0121 - val_loss: 0.0124
Epoch 10/60
6/6 [==============================] - 4s 677ms/step - loss: 0.0115 - val_loss: 0.0108
Epoch 11/60
6/6 [====

In [ ]:
del X_train_autoencoder
del X_test_autoencoder

save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
cnn_autoencoder_right_left.save('/gdrive/My Drive/Models_Static/Right', options=save_locally)

In [ ]:
with strategy.scope():
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    reconstructed_right_left = tf.keras.models.load_model('/gdrive/My Drive/Models_Static/Right', options=load_locally)

In [4]:
  load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
  reconstructed_right_left = tf.keras.models.load_model('/gdrive/My Drive/Yedek/Models_Static/Right', options=load_locally)

In [5]:
image_size = (480,640)
train_ds_left=tf.data.Dataset.load(f"/gdrive/My Drive/Yedek/TF_Datasets_filtered/Left/train")
val_ds_left=tf.data.Dataset.load(f"/gdrive/My Drive/Yedek/TF_Datasets_filtered/Left/val")

X_train_left=np.empty((0,image_size[0],image_size[1],3),dtype='float32')
y_train_left=np.empty((0,),dtype='float32')
X_test_left=np.empty((0,image_size[0],image_size[1],3),dtype='float32')
y_test_left=np.empty((0,),dtype='float32')

for image_batch, labels_batch in train_ds_left:
   X_train_left=np.concatenate((X_train_left,image_batch),axis=0)
   y_train_left=np.concatenate((y_train_left,labels_batch),axis=0)

for image_batch, labels_batch in val_ds_left:
   X_test_left=np.concatenate((X_test_left,image_batch),axis=0)
   y_test_left=np.concatenate((y_test_left,labels_batch),axis=0)


X_train_left=X_train_left / 255.
X_test_left=X_test_left / 255.

y_train=y_train_left
y_test=y_test_left

train_ds_right=tf.data.Dataset.load(f"/gdrive/My Drive/Yedek/TF_Datasets_filtered/Right/train")
val_ds_right=tf.data.Dataset.load(f"/gdrive/My Drive/Yedek/TF_Datasets_filtered/Right/val")

X_train_right=np.empty((0,image_size[0],image_size[1],3),dtype='float32')
y_train_right=np.empty((0,),dtype='float32')
X_test_right=np.empty((0,image_size[0],image_size[1],3),dtype='float32')
y_test_right=np.empty((0,),dtype='float32')

for image_batch, labels_batch in train_ds_right:
   X_train_right=np.concatenate((X_train_right,image_batch),axis=0)
   y_train_right=np.concatenate((y_train_right,labels_batch),axis=0)

for image_batch, labels_batch in val_ds_right:
   X_test_right=np.concatenate((X_test_right,image_batch),axis=0)
   y_test_right=np.concatenate((y_test_right,labels_batch),axis=0)


X_train_right=X_train_right / 255.
X_test_right=X_test_right / 255.


xm_train_left=reconstructed_right_left.encoder(X_train_left)
xm_test_left=reconstructed_right_left.encoder(X_test_left)

xm_train_right=reconstructed_right_left.encoder(X_train_right)
xm_test_right=reconstructed_right_left.encoder(X_test_right)

del X_train_left
del X_train_right
del X_test_left
del X_test_right
del train_ds_left
del val_ds_left
del train_ds_right
del val_ds_right
del reconstructed_right_left


In [6]:
xm_train_left.shape

TensorShape([150, 30, 40, 48])

In [7]:
xm_train_right.shape

TensorShape([150, 30, 40, 48])

In [8]:
xm_train_left=tf.reshape(xm_train_left,[xm_train_left.shape[0],-1])
xm_test_left=tf.reshape(xm_test_left,[xm_test_left.shape[0],-1])

xm_train_right=tf.reshape(xm_train_right,[xm_train_right.shape[0],-1])
xm_test_right=tf.reshape(xm_test_right,[xm_test_right.shape[0],-1])

In [9]:
xm_train_left.shape

TensorShape([150, 57600])

# **Concatenation of left and right vectors**

In [10]:

  xm_train=np.empty((0,xm_train_left.shape[1]*2),dtype='float32')

  xm_test=np.empty((0,xm_test_left.shape[1]*2),dtype='float32')

  for i in range(xm_train_left.shape[0]):

      xm_train_1=np.empty((0,xm_train.shape[1]),dtype='float32')

      xm_train_1=np.concatenate((np.reshape(xm_train_right[i,:],(1,-1)),np.reshape(xm_train_left[i,:],(1,-1))),axis=1)

      xm_train=np.concatenate((xm_train,np.reshape(xm_train_1,(1,-1))),axis=0)

  for j in range(xm_test_left.shape[0]):

      xm_test_1=np.empty((0,xm_test.shape[1]),dtype='float32')

      xm_test_1=np.concatenate((np.reshape(xm_test_right[j,:],(1,-1)),np.reshape(xm_test_left[j,:],(1,-1))),axis=1)

      xm_test=np.concatenate((xm_test,np.reshape(xm_test_1,(1,-1))),axis=0)
'''
xm_train=np.empty((0,xm_train_left.shape[1]),dtype='float32')

xm_test=np.empty((0,xm_test_left.shape[1]),dtype='float32')


for i in range(xm_train_left.shape[0]):

      xm_train_1=np.empty((0,xm_train_left.shape[1]),dtype='float32')

      xm_train_1=np.concatenate((xm_train_1,np.reshape(xm_train_left[i,:],(1,-1))),axis=0)
      xm_train_1=np.concatenate((xm_train_1,np.reshape(xm_train_right[i,:],(1,-1))),axis=0)

      xm_train=np.concatenate((xm_train,np.reshape(xm_train_1.max(axis=0),(1,-1))),axis=0)

for j in range(xm_test_left.shape[0]):

      xm_test_1=np.empty((0,xm_test_left.shape[1]),dtype='float32')

      xm_test_1=np.concatenate((xm_test_1,np.reshape(xm_test_left[j,:],(1,-1))),axis=0)
      xm_test_1=np.concatenate((xm_test_1,np.reshape(xm_test_right[j,:],(1,-1))),axis=0)

      xm_test=np.concatenate((xm_test,np.reshape(xm_test_1.max(axis=0),(1,-1))),axis=0)
'''
del xm_train_1
del xm_test_1
del xm_train_left
del xm_train_right
print(xm_train.shape)
gc.collect()

(150, 115200)


32920

## **Machine Learning**

In [11]:
from sklearn.metrics import classification_report

In [12]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

xm_train=scaler.fit_transform(xm_train)
xm_test=scaler.transform(xm_test)

In [13]:
from sklearn.svm import SVC

svc=SVC(kernel="sigmoid")
svc.fit(xm_train, y_train)

In [14]:
y_pred=svc.predict(xm_test)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm=confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[35  3]
 [10 15]]
              precision    recall  f1-score   support

         0.0       0.78      0.92      0.84        38
         1.0       0.83      0.60      0.70        25

    accuracy                           0.79        63
   macro avg       0.81      0.76      0.77        63
weighted avg       0.80      0.79      0.79        63



In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svc.classes_)
disp.plot()

In [ ]:
y_train=load_hkl['ytrain']
y_test=load_hkl['ytest']